<img style="float: right; margin-right: 100px" src="./images/P404_f2.png" width="200" height="200" />

# HiPSter Inference using Illustris TNG

This notebook demonstrates how to use HiPSter to perform inference on the Illustris TNG simulation
data.



## Hierarchical Progressive Surveys (HiPS)

The Hierarchical Progressive Surveys (HiPS) offer an effective method for visualizing images on a
sphere.
This technique allows for the representation of the sky or a planetary surface on a spherical
format, providing progressively detailed views as you zoom in.
This is achieved through various levels of tiling, where a HiPS tile is split into four new tiles at each level.
Rather than visualizing the sky, we reconstruct the trained dataset using the center of the HiPS tile as the reference point in the spherical latent space.

<img src="./images/HEALPix.png" style="width:50%;">

The task `hips.HiPSGenerator` creates HiPS tiles using the latent space of a trained model. For this
process only the decoder is needed. The `hipster.Inference` utilizes `ONNXRuntime` to execute the
encoding process. The `max_order` parameter specifies the maximum order for which HiPS tiles will be generated.


In [9]:
import hipster

hipster.HiPSGenerator(
    decoder=hipster.Inference("data/illustris/models/full_trained/decoder.onnx"),
    image_maker=hipster.ImagePlotter(),
    max_order=4,
    hips_path="output/illustris",
).execute()

Executing task: HiPSGenerator


## Catalog as VOTable



In [10]:
hipster.VOTableGenerator(
    encoder=hipster.Inference("data/illustris/models/full_trained/encoder.onnx"),
    data_directory="data/illustris/parquet",
    output_file="illustris.vot",
    root_path="output",
).execute()

Executing task: VOTableGenerator


## Visualize HiPS tiles and catalog using Aladin

- [aladin-lite](https://github.com/cds-astro/aladin-lite)
- [ipyaladin](https://github.com/cds-astro/ipyaladin)

In [11]:
from ipyaladin import Aladin

aladin = Aladin(survey="output/illustris", fov=180, show_fullscreen_control=False)
aladin.add_catalog_from_URL("output/illustris.vot", {"source_size": 5, "color": "red"})
aladin

Aladin(survey='output/illustris')

## Command Line Interface (CLI)

In production, the HiPS tiles are generated using the CLI, similar to the PyTorch Lightning CLI.
All configurations are written in a [yaml-file](./configs/hipster/illustris.yaml) and executed with the following command:

In [7]:
!hipster --config configs/hipster/illustris.yaml

Base URL: http://localhost:8083
Tasks:
  - HiPSGenerator
  - VOTableGenerator
Executing task: HiPSGenerator
Executing task: VOTableGenerator
Generating HTML page...


Now, let's run a webserver to visualize Aldin-Lite with the produced HiPS tiles and catalog:

In [8]:
!python -m http.server 8083 --bind localhost --directory HiPSter

Serving HTTP on 127.0.0.1 port 8083 (http://127.0.0.1:8083/) ...
127.0.0.1 - - [27/Apr/2025 15:56:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 15:56:05] code 404, message File not found
127.0.0.1 - - [27/Apr/2025 15:56:05] "GET /illustris/model/Moc.fits HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2025 15:56:05] code 404, message File not found
127.0.0.1 - - [27/Apr/2025 15:56:05] "GET /illustris/model/Norder3/Allsky.jpg HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2025 15:56:08] "GET //thumbnails/TNG100/099/464490.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 15:56:10] "GET //images/TNG100/099/464490.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 15:56:18] "GET //thumbnails/TNG100/099/462241.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 15:56:19] "GET //images/TNG100/099/462241.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 15:56:24] "GET //thumbnails/TNG100/099/462010.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 15:56:26] "GET //images/TNG100/099/462010.jpg HTTP/1.1" 200 -

Keyboard i